# Exploration Data Analisys for World Bank datasets

В ходе данного занятия мы подготовить EDA для датасета **World Bank** и подготовим все необходимое для реализации ETL.

**Данные:**

[World Bank Indicator Data](https://data.worldbank.org/indicator) - Данные социально-экономических индикаторов для стран по всему миру. Как пример индикаторов можно привести популяцию (население).

[World Bank Project Data](https://datacatalog.worldbank.org/dataset/world-bank-projects-operations) - Данные проектов различных стран начиная с 1947 года.

#### В ходе данного занятия мы пройдем следующие этапы:
**Извлечение данных из разных источников:**
- csv
- json
- Базы данных

**Преобразование данных**
- Обхединения данных из различных источников
- Отчистка данных
- Парсинг дат
- Заполнение пропусков
- Работа с дубликатами
- Удаление незначимых переменных
- Выбросы в данных

**Формирование выходной таблицы**
- Построим таблицу, которую можно использовать для обучения моделей

**Цель ноутбука**
- Подготовить несколько гипотез для этапа преобразования данных
- Подготовить скрипты для реализации ETL-пайплайна

## Imports

In [3]:
%load_ext autoreload
%autoreload 2

In [5]:
### pip install -r requirements.txt

In [1]:
#pip install bs4

In [2]:
import os
import sys
import json
from encodings.aliases import aliases
from collections import defaultdict

sys.path.append('../')

import sqlite3
from sklearn.linear_model import LinearRegression
import chardet
import numpy as np
import pandas as pd
from pycountry import countries
import requests
import pandas as pd
from sqlalchemy import create_engine
from bs4 import BeautifulSoup
import matplotlib.pyplot as plt
%matplotlib inline

## Посмотрим на данные

В данном разделе мы откроем все файлы и посмотрим на их содержимое

In [3]:
for dirname, _, filenames in os.walk('data'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

data/preprocess_data/non_countries.json
data/preprocess_data/result_not_end.csv
data/preprocess_data/population_data_2001_2017.csv
data/preprocess_data/projects_data_Country_Date_repair.csv
data/preprocess_data/population_data.db
data/preprocess_data/projects_data.csv
data/preprocess_data/country_not_found_mapping.json
data/preprocess_data/population_data_1960_1980.json


### Рассмотри файл `projects_data.csv`

In [4]:
df_projects = pd.read_csv('data/preprocess_data/projects_data.csv', dtype='str')
print(df_projects.shape)
df_projects.head()

(18248, 24)


,id,regionname,countryname,prodline,lendinginstr,lendinginstrtype,supplementprojectflg,productlinetype,projectstatusdisplay,status,...,lendprojectcost,ibrdcommamt,idacommamt,totalamt,grantamt,url,sector1,sector,mjsector,theme1
0,P162228,Other,World;World,RE,Investment Project Financing,IN,N,L,Active,Active,...,"500,000",0,0,0,"500,000",http://projects.worldbank.org/P162228?lang=en,!$!0,NaN,NaN,!$!0
1,P163962,Africa,Democratic Republic of the Congo;Democratic Re...,PE,Investment Project Financing,IN,N,L,Active,Active,...,"200,000,000",0,"200,000,000","200,000,000",0,http://projects.worldbank.org/P163962?lang=en,!$!0,NaN,NaN,!$!0
2,P167672,South Asia,People's Republic of Bangladesh;People's Repub...,PE,Investment Project Financing,IN,Y,L,Active,Active,...,"50,000,000",0,"58,330,000","58,330,000",0,http://projects.worldbank.org/P167672?lang=en,!$!0,NaN,NaN,!$!0
3,P158768,South Asia,Islamic Republic of Afghanistan;Islamic Repu...,PE,Investment Project Financing,IN,N,L,Active,Active,...,"50,000,000",0,"20,000,000","20,000,000",0,http://projects.worldbank.org/P158768?lang=en,!$!0,NaN,NaN,!$!0
4,P161364,Africa,Federal Republic of Nigeria;Federal Republic o...,PE,Investment Project Financing,IN,N,L,Active,Active,...,"100,000,000",0,"100,000,000","100,000,000",0,http://projects.worldbank.org/P161364?lang=en,Social Protection!$!63!$!SA,Social Protection;Social Protection;Other Indu...,Social Protection;Social Protection;Industry; ...,!$!0


In [5]:
# Считаем количество значений `null` в датафрейме
df_projects.isnull().sum()

id                         0
regionname                 0
countryname                0
prodline                   0
lendinginstr             246
lendinginstrtype         246
supplementprojectflg      53
productlinetype            0
projectstatusdisplay       4
status                     4
project_name               0
boardapprovaldate       1504
board_approval_month    1504
closingdate             3349
lendprojectcost          125
ibrdcommamt                0
idacommamt                 0
totalamt                   0
grantamt                   0
url                        0
sector1                    0
sector                   638
mjsector                 638
theme1                     0
dtype: int64

Очевидно, что есть большое количество пропущенных значений. Так например можно заметить, что в поле `countryname` значений не пропущено, а аналогичное поле `country` имеет 14045 пропущенных значений.

In [6]:
columns = df_projects.columns
for column in columns:
    if df_projects.isnull().sum()[column] / df_projects.shape[0] > 0.2:
        df_projects.drop(column, axis=1, inplace=True)

In [7]:
df_projects.isnull().sum()

id                         0
regionname                 0
countryname                0
prodline                   0
lendinginstr             246
lendinginstrtype         246
supplementprojectflg      53
productlinetype            0
projectstatusdisplay       4
status                     4
project_name               0
boardapprovaldate       1504
board_approval_month    1504
closingdate             3349
lendprojectcost          125
ibrdcommamt                0
idacommamt                 0
totalamt                   0
grantamt                   0
url                        0
sector1                    0
sector                   638
mjsector                 638
theme1                     0
dtype: int64

Сохраним данный файл

In [8]:
df_projects.to_csv('data/preprocess_data/projects_data.csv', index=False)

### Рассмотрим файл `df_population.csv`

In [ ]:
df_population = pd.read_csv('data/clear_data/population_data.csv', dtype=str)
print(df_population.shape)
df_population.head()

In [ ]:
df_population = df_population.drop('Unnamed: 62', axis=1)

In [ ]:
df_population.columns

In [ ]:
columns_1 = [
    'Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
    '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
    '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
    '1978', '1979', '1980'
]
columns_2  =  [
    'Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
    '1981', '1982', '1983', '1984', '1985', '1986',
    '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
    '1996', '1997', '1998', '1999', '2000'
]
columns_3   = [
    'Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
    '2001', '2002', '2003', '2004',
    '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
    '2014', '2015', '2016', '2017'
]

In [ ]:
df_population[columns_1].to_json('data/preprocess_data/population_data_1960_1980.json', orient='records')

In [ ]:
engine_sqlite3 = create_engine('sqlite:///data/preprocess_data/population_data.db')
engine_sqlite3.raw_connection()
df_population[columns_2].to_sql('population_data_1981_2000', engine_sqlite3, if_exists='replace', index=False)

In [ ]:
engine_sqlite3.table_names()

In [ ]:
df_population[columns_3].to_csv('data/preprocess_data/population_data_2001_2017.csv', index=False)

In [ ]:
df_indicator = pd.read_csv('data/clear_data/population_data.csv')
df_indicator.drop(['Unnamed: 62'], axis=1, inplace=True)

df_projects = pd.read_csv('data/preprocess_data/projects_data.csv', dtype=str)

In [ ]:
print(df_projects.shape)
df_projects.head()

In [ ]:
df_indicator[['Country Name', 'Country Code']].drop_duplicates()

## Работа с полем `Country`
Стоит обратить внимание, что в файле `df_projects`, в столбце 'Country' значения не заполнены. 

Постановка проблемы:
- Файл `df_projects` содержит абревиатуры стран, в то время как файл `df_indicator` содержит названия и абревиатуры.

Намного проще объединять файлы с уникальными идентификарами. Поэтому приведем данные к одному виду.

In [ ]:
df_projects['countryname'].unique()

In [ ]:
df_projects['Official Country Name'] = df_projects['countryname'].str.split(';').str.get(0)

In [ ]:
country_not_found = [] 
project_country_abbrev_dict = defaultdict(str) 

# Итерируемся по всем возможным названиям стран
for country in df_projects['Official Country Name'].drop_duplicates().sort_values():
    try: 
        # Сопоставляем значениям ключа - уникальный код ISO
        project_country_abbrev_dict[country] = countries.lookup(country).alpha_3
    except:
        # Если такого кода нет, 
        print(country, 'country_not_found')
        country_not_found.append(country)

In [ ]:
# Проверим есть ли название в файле df_projects
indicator_countries = df_indicator[['Country Name', 'Country Code']].drop_duplicates().sort_values(by='Country Name')

for country in country_not_found:
    if country in indicator_countries['Country Name'].tolist():
        print(country)

In [ ]:
country_not_found_mapping = {
    'Co-operative Republic of Guyana': 'GUY',
    'Commonwealth of Australia':'AUS',
    'Democratic Republic of Sao Tome and Prin':'STP',
    'Democratic Republic of the Congo':'COD',
    'Democratic Socialist Republic of Sri Lan':'LKA',
    'East Asia and Pacific':'EAS',
    'Europe and Central Asia': 'ECS',
    'Islamic  Republic of Afghanistan':'AFG',
    'Latin America':'LCN',
    'Caribbean':'LCN',
    'Macedonia':'MKD',
    'Middle East and North Africa':'MEA',
    'Oriental Republic of Uruguay':'URY',
    'Republic of Congo':'COG',
    "Republic of Cote d'Ivoire":'CIV',
    'Republic of Korea':'KOR',
    'Republic of Niger':'NER',
    'Republic of Kosovo':'XKX',
    'Republic of Rwanda':'RWA',
    'Republic of The Gambia':'GMB',
    'Republic of Togo':'TGO',
    'Republic of the Union of Myanmar':'MMR',
    'Republica Bolivariana de Venezuela':'VEN',
    'Sint Maarten':'SXM',
    "Socialist People's Libyan Arab Jamahiriy":'LBY',
    'Socialist Republic of Vietnam':'VNM',
    'Somali Democratic Republic':'SOM',
    'South Asia':'SAS',
    'St. Kitts and Nevis':'KNA',
    'St. Lucia':'LCA',
    'St. Vincent and the Grenadines':'VCT',
    'State of Eritrea':'ERI',
    'The Independent State of Papua New Guine':'PNG',
    'West Bank and Gaza':'PSE',
    'World':'WLD',
}

In [ ]:
project_country_abbrev_dict.update(country_not_found_mapping)

In [ ]:
df_projects['Country Code'] = df_projects['Official Country Name'].apply(lambda x: project_country_abbrev_dict[x])

In [ ]:
df_projects = df_projects[df_projects['Country Code'] != '']

## Обработка дат

In [ ]:
df_projects.head(5)[['boardapprovaldate', 'board_approval_month', 'closingdate']]

In [ ]:
df_projects['boardapprovaldate'] = pd.to_datetime(df_projects['boardapprovaldate'])
df_projects['closingdate'] = pd.to_datetime(df_projects['closingdate'])

In [ ]:
df_projects.fillna('', inplace=True)

In [ ]:
df_projects['approvalyear'] = pd.to_numeric(df_projects['boardapprovaldate'].dt.year)
df_projects['approvalday'] = pd.to_numeric(df_projects['boardapprovaldate'].dt.year)
df_projects['approvalweekday'] = pd.to_numeric(df_projects['boardapprovaldate'].dt.year)
df_projects['closingyear'] = pd.to_numeric(df_projects['closingdate'].dt.year)
df_projects['closingday'] = pd.to_numeric(df_projects['closingdate'].dt.year)
df_projects['closingweekday'] = pd.to_numeric(df_projects['closingdate'].dt.year)

# df_projects.drop(columns=['boardapprovaldate',  'closingdate'], inplace=True)

In [ ]:
df_projects

In [ ]:
df_projects.to_csv('data/preprocess_data/projects_data_Country_Date_repair.csv', index=False)

#### Почему некоторые поля `closingdate` могут быть пустыми?

## Заполнение данных

Прощенные значения

In [ ]:
df_vvp = pd.read_csv('data/clear_data/gdp_data.csv')
df_vvp.drop('Unnamed: 62', axis=1, inplace=True)

In [ ]:
print(df_vvp.shape)
df_vvp.head()

In [ ]:
df_vvp.isnull().sum()

In [ ]:
# put the data set into long form instead of wide
df_vvp_melt = pd.melt(df, id_vars=['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code'], var_name='year', value_name='GDP')

def plot_results(column_name):
    # plot the results for Afghanistan, Albania, and Honduras
    fig, ax = plt.subplots(figsize=(8,6))

    df_vvp_melt[(df_vvp_melt['Country Name'] == 'Brazil') | 
            (df_vvp_melt['Country Name'] == 'Russian Federation') | 
            (df_vvp_melt['Country Name'] == 'India')].groupby('Country Name').plot('year', column_name, legend=True, ax=ax)
    ax.legend(labels=['Brazil', 'Russian Federation', 'India'])
    
plot_results('GDP')

In [ ]:
df_vvp_melt['GDP_filled'] = df_vvp_melt.groupby('Country Name')['GDP'].transform(lambda x: x.fillna(x.mean()))

In [ ]:
plot_results('GDP_filled')

In [ ]:
df_vvp_melt['GDP_ff_bf'] = df_vvp_melt.sort_values('year').groupby('Country Name')['GDP'].fillna(method='ffill').fillna(method='bfill')

In [ ]:
plot_results('GDP_ff_bf')

In [ ]:
df_vvp_melt['GDP_ff_bf'].isnull().sum()

In [ ]:
df_vvp_melt.head()

## Переменные экономического сектора

Рассмотрим поле `sector1` и узнаем как изменить его, чтобы данные были чистыми

In [9]:
# Create a list of the unique values in sector1. Use the sort_values() and unique() pandas methods. 
# And then convert those results into a Python list
uniquesectors1 = df_projects['sector1'].sort_values().unique()
uniquesectors1

array(['!$!0', '(Historic)Agency reform!$!50!$!AG',
       '(Historic)Agricultural credit!$!100!$!AC', ...,
       'Workforce Development and Vocational Education!$!96!$!EW',
       'Workforce Development and Vocational Education!$!98!$!EW',
       'Workforce Development and Vocational Education!$!99!$!EW'],
      dtype=object)

In [10]:
# run this code cell to see the number of unique values
print('Number of unique values in sector1:', len(uniquesectors1))

Number of unique values in sector1: 3060


С этой переменной «сектор1» есть несколько проблем. Во-первых, есть значения с меткой '!$!0'. Их следует заменить на NaN.

Более того, каждое значение сектора1 заканчивается строкой из десяти или одиннадцати символов, например '!$!49!$!EP'. Некоторые сектора отображаются в списке дважды, например:
 «Другая промышленность; Торговля и услуги!$!70!$!ЫЗ',
 «Другая промышленность; Торговля и услуги!$!63!$!YZ',

Но похоже, что это на самом деле один и тот же сектор. Вам нужно будет удалить все, что находится за восклицательным знаком.

Многие значения переменной сектор1 начинаются с термина «(Исторический)». Попробуйте удалить и эту фразу.

### replace() method

В pandas вы можете использовать метод replace() для поиска текста и замены частей строки другой строкой. Если вы знаете точную строку, которую ищете, метод replace() не вызывает затруднений. Например, предположим, что вы хотите удалить строку «(Пробная версия)» из этих данных:

| data                     |
|--------------------------|
| '(Trial) Banking'        |
| 'Banking'                |
| 'Farming'                |
| '(Trial) Transportation' |

Вы можете использовать `df['data'].replace('(Trial'), '')` для замены (Trial) пустой строкой.


In [ ]:
df_projects['sector1'] = df_projects['sector1'].replace('!$!0', np.nan)

df_projects['sector1'] = df_projects['sector1'].replace('!.+', '', regex=True)

df_projects['sector1'] = df_projects['sector1'].replace('^(\(Historic\))', '', regex=True)

print('Number of unique sectors after cleaning:', len(list(df_projects['sector1'].unique())))
print('Percentage of null values after cleaning:', 100 * df_projects['sector1'].isnull().sum() / df_projects['sector1'].shape[0])

Сейчас существует 156 уникальных категориальных значений. Это лучше, чем 3060.

> ## Поиск выбросов

Рассмотрим набор данных Всемирного банка по ВВП и населению. Сначала посмотрим на данные с одномерной точки зрения, а затем с двухмерной точки зрения.

In [ ]:
df_vvp.head()

In [ ]:
# read in the projects data set and do basic wrangling 
gdp = pd.read_csv('data/clear_data/gdp_data.csv')
gdp.drop(['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1)
population = pd.read_csv('data/clear_data/population_data.csv')
population.drop(['Unnamed: 62', 'Country Code', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1)


# Reshape the data sets so that they are in long format
gdp_melt = gdp.melt(id_vars=['Country Name'], 
                    var_name='year', 
                    value_name='gdp')

# Use back fill and forward fill to fill in missing gdp values
gdp_melt['gdp'] = gdp_melt.sort_values('year').groupby('Country Name')['gdp'].fillna(method='ffill').fillna(method='bfill')

population_melt = population.melt(id_vars=['Country Name'], 
                                  var_name='year', 
                                  value_name='population')

# Use back fill and forward fill to fill in missing population values
population_melt['population'] = population_melt.sort_values('year').groupby('Country Name')['population'].fillna(method='ffill').fillna(method='bfill')

# merge the population and gdp data together into one data frame
df_country = gdp_melt.merge(population_melt, on=('Country Name', 'year'))

# filter data for the year 2016
df_2016 = df_country[df_country['year'] == '2016']

# see what the data looks like
df_2016.head(10)

Explore the data set to identify outliers using the Tukey rule.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline 

# Построим boxplot для данных популяции за 2016 год
df_2016.plot('population', kind='box')

# Построим boxplot для данных ввп за 2016 год
df_2016.plot('gdp', kind='box')

In [ ]:
# Удалим строки из данных, которые имеют значения `CountryName` в списке non_countries
# Сохраняем результаты фильтра обратно в переменную df_2016

non_countries = ['World',
 'High income',
 'OECD members',
 'Post-demographic dividend',
 'IDA & IBRD total',
 'Low & middle income',
 'Middle income',
 'IBRD only',
 'East Asia & Pacific',
 'Europe & Central Asia',
 'North America',
 'Upper middle income',
 'Late-demographic dividend',
 'European Union',
 'East Asia & Pacific (excluding high income)',
 'East Asia & Pacific (IDA & IBRD countries)',
 'Euro area',
 'Early-demographic dividend',
 'Lower middle income',
 'Latin America & Caribbean',
 'Latin America & the Caribbean (IDA & IBRD countries)',
 'Latin America & Caribbean (excluding high income)',
 'Europe & Central Asia (IDA & IBRD countries)',
 'Middle East & North Africa',
 'Europe & Central Asia (excluding high income)',
 'South Asia (IDA & IBRD)',
 'South Asia',
 'Arab World',
 'IDA total',
 'Sub-Saharan Africa',
 'Sub-Saharan Africa (IDA & IBRD countries)',
 'Sub-Saharan Africa (excluding high income)',
 'Middle East & North Africa (excluding high income)',
 'Middle East & North Africa (IDA & IBRD countries)',
 'Central Europe and the Baltics',
 'Pre-demographic dividend',
 'IDA only',
 'Least developed countries: UN classification',
 'IDA blend',
 'Fragile and conflict affected situations',
 'Heavily indebted poor countries (HIPC)',
 'Low income',
 'Small states',
 'Other small states',
 'Not classified',
 'Caribbean small states',
 'Pacific island small states']

df_2016 = df_2016[~df_2016['Country Name'].isin(non_countries)]

>### 2-мерный анализ

Затем посмотрите на данные с двухмерной точки зрения.

В следующей ячейке кода представлены данные о ВВП и населении, включая название страны в каждой точке.

In [ ]:
x = list(df_2016['population'])
y = list(df_2016['gdp'])
text = df_2016['Country Name']

fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(x, y)
plt.title('GDP vs Population')
plt.xlabel('population')
plt.ylabel('GDP')
for i, txt in enumerate(text):
    ax.annotate(txt, (x[i],y[i]))

В США, Китае и Индии значения настолько велики, что остальные данные трудно рассмотреть.

In [ ]:
df_no_large = (df_2016['Country Name'] != 'United States') & (df_2016['Country Name'] != 'India') & (df_2016['Country Name'] != 'China')
x = list(df_2016[df_no_large]['population'])
y = list(df_2016[df_no_large]['gdp'])
text = df_2016[df_no_large]['Country Name']

fig, ax = plt.subplots(figsize=(15,10))
ax.scatter(x, y)
plt.title('GDP vs Population')
plt.xlabel('population')
plt.ylabel('GDP')
for i, txt in enumerate(text):
    ax.annotate(txt, (x[i],y[i]))

# Load 

In this part, I'll load data into different formats: a csv file, a json file, and a SQLite database.

In [ ]:
gdp

In [ ]:
gdp['Country Name']

In [ ]:
population

In [ ]:
df_projects

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline 

# read in the projects data set and do basic wrangling 
gdp = pd.read_csv('data/clear_data/gdp_data.csv')
gdp.drop(['Unnamed: 62', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1)
population = pd.read_csv('data/clear_data/population_data.csv')
population.drop(['Unnamed: 62', 'Indicator Name', 'Indicator Code'], inplace=True, axis=1)


# Reshape the data sets so that they are in long format
gdp_melt = gdp.melt(id_vars=['Country Name', 'Country Code'], 
                    var_name='year',
                    value_name='gdp')

# Use back fill and forward fill to fill in missing gdp values
gdp_melt['gdp'] = gdp_melt.sort_values('year').groupby(['Country Name', 'Country Code'])['gdp'].fillna(method='ffill').fillna(method='bfill')

population_melt = population.melt(id_vars=['Country Name', 'Country Code'], 
                                  var_name='year', 
                                  value_name='population')

# Use back fill and forward fill to fill in missing population values
population_melt['population'] = population_melt.sort_values('year').groupby('Country Name')['population'].fillna(method='ffill').fillna(method='bfill')

# merge the population and gdp data together into one data frame
df_indicator = gdp_melt.merge(population_melt, on=('Country Name', 'Country Code', 'year'))


# remove non countries from the data
df_indicator  = df_indicator[~df_indicator['Country Name'].isin(non_countries)]
df_indicator.reset_index(inplace=True, drop=True)

df_indicator.columns = ['countryname', 'countrycode', 'year', 'gdp', 'population']

# output the first few rows of the data frame
df_indicator.head()

In [ ]:
df_indicator

Run this code cell to read in the countries data set. This will create a data frame called df_projects containing the World Bank projects data. The data frame only has the 'id', 'countryname', 'countrycode', 'totalamt', and 'year' columns.

In [ ]:
# read in the projects data set with all columns type string
df_projects = pd.read_csv('data/preprocess_data/projects_data_Country_Date_repair.csv', dtype=str)


df_projects['countryname'] = df_projects['countryname'].str.split(';').str.get(0)

# set up the libraries and variables

country_not_found = [] # stores countries not found in the pycountry library
project_country_abbrev_dict = defaultdict(str) # set up an empty dictionary of string values

# iterate through the country names in df_projects. 
# Create a dictionary mapping the country name to the alpha_3 ISO code
for country in df_projects['countryname'].drop_duplicates().sort_values():
    try: 
        # look up the country name in the pycountry library
        # store the country name as the dictionary key and the ISO-3 code as the value
        project_country_abbrev_dict[country] = countries.lookup(country).alpha_3
    except:
        # If the country name is not in the pycountry library, then print out the country name
        # And store the results in the country_not_found list
        country_not_found.append(country)
        
# run this code cell to load the dictionary

country_not_found_mapping = {'Co-operative Republic of Guyana': 'GUY',
             'Commonwealth of Australia':'AUS',
             'Democratic Republic of Sao Tome and Prin':'STP',
             'Democratic Republic of the Congo':'COD',
             'Democratic Socialist Republic of Sri Lan':'LKA',
             'East Asia and Pacific':'EAS',
             'Europe and Central Asia': 'ECS',
             'Islamic  Republic of Afghanistan':'AFG',
             'Latin America':'LCN',
              'Caribbean':'LCN',
             'Macedonia':'MKD',
             'Middle East and North Africa':'MEA',
             'Oriental Republic of Uruguay':'URY',
             'Republic of Congo':'COG',
             "Republic of Cote d'Ivoire":'CIV',
             'Republic of Korea':'KOR',
             'Republic of Niger':'NER',
             'Republic of Kosovo':'XKX',
             'Republic of Rwanda':'RWA',
              'Republic of The Gambia':'GMB',
              'Republic of Togo':'TGO',
              'Republic of the Union of Myanmar':'MMR',
              'Republica Bolivariana de Venezuela':'VEN',
              'Sint Maarten':'SXM',
              "Socialist People's Libyan Arab Jamahiriy":'LBY',
              'Socialist Republic of Vietnam':'VNM',
              'Somali Democratic Republic':'SOM',
              'South Asia':'SAS',
              'St. Kitts and Nevis':'KNA',
              'St. Lucia':'LCA',
              'St. Vincent and the Grenadines':'VCT',
              'State of Eritrea':'ERI',
              'The Independent State of Papua New Guine':'PNG',
              'West Bank and Gaza':'PSE',
              'World':'WLD'}

project_country_abbrev_dict.update(country_not_found_mapping)

df_projects['countrycode'] = df_projects['countryname'].apply(lambda x: project_country_abbrev_dict[x])

df_projects['boardapprovaldate'] = pd.to_datetime(df_projects['boardapprovaldate'])

df_projects['year'] = df_projects['boardapprovaldate'].dt.year.astype(str).str.slice(stop=4)

df_projects['totalamt'] = pd.to_numeric(df_projects['totalamt'].str.replace(',',""))

df_projects = df_projects[['id', 'countryname', 'countrycode', 'totalamt', 'year']]

df_projects.head()

In [ ]:
df_merged = df_projects.merge(df_indicator, how='left', on=['countrycode', 'year'])

In [ ]:
df_merged[(df_merged['year'] == '2017') & (df_merged['countryname_y'] == 'Jordan')]

In [ ]:
df_merged.to_csv('data/preprocess_data/result_not_end.csv', index=False)

> ## Немного выводов

1. Данные следует почистить от стран, которые не являются странами (например: `World`)
2. Нужно обработать тип данных `DateTime`
3. Также необходимо перевести числовые значения формата `103,101` в формат `103101`
4. Исходя из результатов построим финальную таблицу в которой будут содержаться по годам данные разных экономических и макроэкономических параметров стран